In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
import os
import sys

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [5]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score
import math
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Importing the dataset
Features = pd.read_csv('/content/drive/MyDrive/DataMining/Datasets/Selected_Features.csv')
X = Features.iloc[: ,:-1].values
Y = Features['class'].values


In [6]:
# # As this is a multiclass classification problem onehotencoding our Y.
# encoder = OneHotEncoder()
# Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [7]:
# splitting data
smote_on_1 = 212
smote = SMOTE(sampling_strategy={'Minimal': smote_on_1,'Moderate': smote_on_1, 'Extreme': smote_on_1})
X, Y = smote.fit_resample(X, Y)
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((636, 26), (636, 4), (212, 26), (212, 4))

In [8]:
# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((636, 26), (636, 4), (212, 26), (212, 4))

In [9]:
# making our data compatible to model.
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((636, 26, 1), (636, 4), (212, 26, 1), (212, 4))

In [10]:
# # # making our data compatible to model.
# # x_train = np.expand_dims(x_train, axis=2)
# # x_test = np.expand_dims(x_test, axis=2)
# y_train = np.expand_dims(y_train, axis=1)
# x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [11]:
import tensorflow as tf


In [12]:
model=Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.3))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.3))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.3))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
# model.add(Dropout(0.3))

model.add(Dense(units=4, activation='softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 26, 256)           1536      
                                                                 
 dropout (Dropout)           (None, 26, 256)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 26, 256)           327936    
                                                                 
 dropout_1 (Dropout)         (None, 26, 256)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 26, 128)           163968    
                                                                 
 dropout_2 (Dropout)         (None, 26, 128)           0         
                                                                 
 conv1d_3 (Conv1D)           (None, 26, 64)            4

In [13]:
# rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
# history=model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test), callbacks=[rlrp])
history=model.fit(x_train, y_train, batch_size=8, epochs=200, validation_data=(x_test, y_test))

Epoch 1/200
80/80 [==============================] - 5s 41ms/step - loss: 1.1901 - accuracy: 0.4560 - val_loss: 1.0901 - val_accuracy: 0.4906
Epoch 2/200
80/80 [==============================] - 3s 35ms/step - loss: 1.1012 - accuracy: 0.5031 - val_loss: 1.1205 - val_accuracy: 0.5000
Epoch 3/200
80/80 [==============================] - 3s 36ms/step - loss: 1.0659 - accuracy: 0.5330 - val_loss: 1.0409 - val_accuracy: 0.5094
Epoch 4/200
80/80 [==============================] - 3s 43ms/step - loss: 1.0297 - accuracy: 0.5000 - val_loss: 1.0177 - val_accuracy: 0.5236
Epoch 5/200
80/80 [==============================] - 3s 38ms/step - loss: 1.0028 - accuracy: 0.5503 - val_loss: 0.9998 - val_accuracy: 0.5236
Epoch 6/200
80/80 [==============================] - 3s 36ms/step - loss: 0.9827 - accuracy: 0.5613 - val_loss: 0.9762 - val_accuracy: 0.5708
Epoch 7/200
80/80 [==============================] - 3s 37ms/step - loss: 0.9733 - accuracy: 0.5770 - val_loss: 0.9782 - val_accuracy: 0.5519
Epoch 

In [14]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


In [20]:
yhat_probs = model.predict(x_test, verbose=0)
yhat_probs

array([[1.00000000e+00, 2.37344391e-22, 2.31887635e-24, 1.71256774e-16],
       [7.39076811e-17, 1.00000000e+00, 3.79235586e-25, 2.76019548e-21],
       [2.59773092e-08, 1.98038430e-09, 9.99030709e-01, 9.69246670e-04],
       [6.17847177e-15, 7.35760108e-03, 9.89091992e-01, 3.55032110e-03],
       [9.99975324e-01, 2.47172029e-05, 1.19774147e-29, 1.87922833e-08],
       [9.99997973e-01, 5.09250100e-08, 1.23825124e-17, 2.03303284e-06],
       [2.34029629e-24, 9.99894500e-01, 6.86353451e-05, 3.68033943e-05],
       [1.26670580e-15, 1.00000000e+00, 9.61897767e-20, 3.80155551e-09],
       [4.25582874e-20, 3.74303758e-01, 6.25695884e-01, 3.76403534e-07],
       [1.23437244e-13, 4.72903956e-08, 8.41266093e-15, 1.00000000e+00],
       [6.43618107e-01, 1.78010881e-01, 1.57300874e-05, 1.78355336e-01],
       [2.01064118e-04, 9.78236973e-01, 2.73424547e-11, 2.15620380e-02],
       [1.00000000e+00, 4.28285046e-18, 2.96033820e-24, 8.03414668e-10],
       [1.00000000e+00, 1.28982229e-24, 0.00000000e

In [18]:
yhat_classes = model.predict_classes(x_test, verbose=0)

AttributeError: ignored